### Antworten

***on macOS***

- start:
 
    ```SQL
    sqlite3 DBPraktikum02.db
    .read creates.sql
    .read inserts.sql
    .mode table
    ```

---

1. Alle Informationen über alle Banken in der Tabelle Bank

    ```SQL
    SELECT * FROM Bank;
    ```

---

2. Alle Informationen ¨uber die Banken mit Geschäftssitz Hamburg in der Tabelle Bank

    ```SQL
    SELECT * FROM Bank WHERE geschaeftssitz = 'Hamburg';
    ```

---

3. Die Namen und Wohnorte der Mitarbeiter, die ab dem 28.4.1998 angestellt wurden, geordnet nach Namen

    ```SQL
    SELECT mname, wohnort FROM Mitarbeiter WHERE anstellungsdatum >= 1998-04-28 ORDER BY mname ASC;
    ```
---

4. Join in zwei Notationen (Komma-Schreibweise und Join-Schreibweise): Die Namen und Wohnorte der Bank-Mitarbeiter, mit dem Namen und der Bankleitzahl ihrer Bank, aber nur, wenn die Mitarbeiter ein niedriges Monatsgehalt (< 2500 EUR) haben. Auf die Join-Bedingung achten!
    1. Komma-Schreibweise

    ```SQL
    SELECT Mitarbeiter.mname AS mitarbeiter_name, Mitarbeiter.wohnort, Bank.bname AS bank_name, Bank.blz AS bankleitzahl
    FROM Mitarbeiter, Bank
    WHERE Mitarbeiter.arbeitgeber_bank = Bank.bname
    AND Mitarbeiter.monatsgehalt < 2500;
    ```

    2. Join-Schreibweise

    ```SQL
    SELECT Mitarbeiter.mname AS mitarbeiter_name, Mitarbeiter.wohnort, Bank.bname AS bank_name, Bank.blz AS bankleitzahl
    FROM Mitarbeiter
    JOIN Bank
    ON Mitarbeiter.arbeitgeber_bank = Bank.bname
    WHERE Mitarbeiter.monatsgehalt < 2500;
    ```
---

5.  Join ¨uber drei Tabellen: Die Nummern und Arten der Konten, die Kunden unterhalten, die durch den Mitarbeiter Olafson betreut werden
```SQL
    SELECT Kunde.betreuer, unterhaelt.kontonummer, Konto.kontoart
    FROM Kunde
    JOIN unterhaelt
    ON Kunde.kname = unterhaelt.inhaber
    JOIN Konto
    ON unterhaelt.kontonummer = Konto.kontonr
    WHERE Kunde.betreuer = 'Olafson';
```

---


6. Riesenjoin 1: Die Besitzer, Nummern und Guthaben der Tagesgeldkonten, die Kunden unterhalten, die durch Mitarbeiter von Frankfurter Banken betreut werden, geordnet nach den Kontobesitzern, bei gleichem Kontobesitzer nach Kontonummer

    ```SQL
    SELECT Kunde.kname AS besitzer, unterhaelt.kontonummer, Konto.guthaben
    FROM Kunde
    JOIN unterhaelt
    ON Kunde.kname = unterhaelt.inhaber
    JOIN Konto
    ON unterhaelt.kontonummer = Konto.kontonr
    JOIN Mitarbeiter
    ON Kunde.betreuer = Mitarbeiter.mname
    JOIN Bank 
    ON Mitarbeiter.arbeitgeber_bank = Bank.bname
    WHERE Bank.geschaeftssitz = 'Frankfurt' AND Konto.kontoart = 'Tagesgeld'
    ORDER BY Kunde.kname ASC, unterhaelt.kontonummer ASC;

    ```

---


7. Riesenjoin 2: Die Besitzer, Nummern und Guthaben der Tagesgeldkonten, die Kunden unterhalten, die bei Filialen von Frankfurter Banken geführt werden, wiederum geordnet nach den Kontobesitzern, bei gleichem Kontobesitzer nach Kontonummer

    ```SQL
    SELECT unterhaelt.inhaber, Konto.kontonr, Konto.guthaben
    FROM Konto
    JOIN unterhaelt 
    ON unterhaelt.kontonummer = Konto.kontonr
    JOIN Kunde 
    ON unterhaelt.inhaber = Kunde.kname and Kunde.geburtsdatum = unterhaelt.inhaber_gebdat
    JOIN Filiale 
    ON Kunde.zustaendige_filiale = Filiale.fname
    JOIN bank 
    ON Filiale.mutterbank = Bank.bname
    WHERE Bank.bname = 'RosaBank' and Konto.kontoart='Tagesgeld'
    ORDER BY unterhaelt.inhaber, Konto.kontonr;
    ```

---

8. Die Anzahl der Mitarbeiter aus Hamburg

    ```SQL
    SELECT COUNT(*) FROM Mitarbeiter WHERE Mitarbeiter.wohnort='Hamburg';
    ```

---

9.  Die Namen der Mitarbeiter mit überdurchschnittlichem Monatsgehalt, und ihre Monatsgehälter

    ```SQL
    SELECT mname FROM Mitarbeiter WHERE (SELECT AVG(monatsgehalt) from Mitarbeiter) < monatsgehalt;
    ```

---

10.  Geschachteltes Select: Die Namen der Mitarbeiter und ihrer Bank, die das folgende Kriterium erfüllen: sie betreuen Kunden aus Leipzig

```SQL
SELECT mname, arbeitgeber_bank
FROM Mitarbeiter
WHERE EXISTS (SELECT betreuer FROM Kunde WHERE wohnort = 'Leipzig' AND mname=betreuer);
```

---

11.  Zwei äquivalente Selects: ein geschachteltes Select und ein Select mit einem Join ¨uber zwei Tabellen: Die Kontonummern und Guthaben aller Girokonten der Filialen der BlackBank, absteigend geordnet nach Kontonummer

```SQL
SELECT Konto.kontonr, Konto.guthaben 
FROM Konto
JOIN Filiale ON Konto.anlage_filiale = Filiale.fname
WHERE Filiale.mutterbank = 'BlackBank' AND Konto.kontoart = 'Giro'
ORDER BY Konto.kontonr DESC;
```

---

```SQL
SELECT Konto.kontonr, Konto.guthaben 
FROM Konto 
WHERE Konto.kontoart = 'Giro'
AND Konto.anlage_filiale IN (SELECT Filiale.fname FROM Filiale WHERE Filiale.mutterbank = 'BlackBank') 
ORDER BY Konto.kontonr DESC;
```

